In [1]:
import re, json

In [3]:

import json

file_path = 'train.json'


with open(file_path, 'r') as file:
        
    json_objs = json.load(file)
    
import jsonlines

edatas = []

for json_obj in json_objs:

    edata = {}

    abstract = ' '.join(json_obj['abstract']).replace('\n',' ')
    lay = ' '.join(json_obj['summary']).replace('\n',' ')

    assert len(json_obj["sections"]) == len(json_obj['headings'])

    cur_articles = []

    for nid, (article_list, heading) in enumerate(zip(json_obj["sections"],json_obj['headings'])):

        cur_articles.append(str(nid+1) + ' ' + heading + ' ' + ' '.join(article_list).replace('\n',' '))

    edata['doi'] = json_obj['id']
    edata['abstract'] = abstract
    edata['plain language summary'] = lay
    edata['article'] = ' '.join(cur_articles)

    edatas.append(edata)


with jsonlines.open('train_elife.jsonl', 'w') as writer:
    for edata in edatas:
        writer.write(edata)

In [126]:
elife_datas = []
with open('./train_elife.jsonl', 'r', encoding="utf-8") as f:
    lines = f.readlines()
    # print(len(lines)) 26214
    for lid in range(len(lines)):
        data = json.loads(lines[lid])
        elife_datas.append(data)

In [127]:
len(elife_datas)

4346

In [24]:
elife_datas[0]

{'doi': 'elife-35500-v1',
 'abstract': 'In temperate climates , winter deaths exceed summer ones . However , there is limited information on the timing and the relative magnitudes of maximum and minimum mortality , by local climate , age group , sex and medical cause of death . We used geo-coded mortality data and wavelets to analyse the seasonality of mortality by age group and sex from 1980 to 2016 in the USA and its subnational climatic regions . Death rates in men and women ≥ 45 years peaked in December to February and were lowest in June to August , driven by cardiorespiratory diseases and injuries . In these ages , percent difference in death rates between peak and minimum months did not vary across climate regions , nor changed from 1980 to 2016 . Under five years , seasonality of all-cause mortality largely disappeared after the 1990s . In adolescents and young adults , especially in males , death rates peaked in June/July and were lowest in December/January , driven by injury 

In [ ]:
plos_datas = []
with open('./train_plos.jsonl', 'r', encoding="utf-8") as f:
    lines = f.readlines()
    # print(len(lines)) 26214
    for lid in range(len(lines)):
        data = json.loads(lines[lid])
        plos_datas.append(data)

In [41]:
len(plos_datas)

26124

In [16]:
plos_datas[0]

{'doi': '10.1371/journal.pntd.0007210',
 'title': 'Evaluation of a novel West Nile virus transmission control strategy that targets Culex tarsalis with endectocide-containing blood meals',
 'abstract': 'Control of arbovirus transmission remains focused on vector control through application of insecticides directly to the environment. However, these insecticide applications are often reactive interventions that can be poorly-targeted, inadequate for localized control during outbreaks, and opposed due to environmental and toxicity concerns. In this study, we developed endectocide-treated feed as a systemic endectocide for birds to target blood feeding Culex tarsalis, the primary West Nile virus (WNV) bridge vector in the western United States, and conducted preliminary tests on the effects of deploying this feed in the field. In lab tests, ivermectin (IVM) was the most effective endectocide tested against Cx. tarsalis and WNV-infection did not influence mosquito mortality from IVM. Chick

In [212]:

elife_prompt = {
    "prompt_input_pls": "Please give me a layman summary of the following paper from eLife dataset.\nBelow is a paper.\n{context}\nNow the paper ends.",
    "prompt_input_exp": "Please give me an expert summary of the following paper from eLife dataset.\nBelow is a paper.\n{context}\nNow the paper ends."
}

plos_prompt = {
    "prompt_input_pls": "Please give me a layman summary of the following paper from PLOS dataset.\nBelow is a paper.\n{context}\nNow the paper ends.",
    "prompt_input_exp": "Please give me an expert summary of the following paper from PLOS dataset.\nBelow is a paper.\n{context}\nNow the paper ends."
}


elife_query_prompt = {
    "prompt_input_pls": "Please answer one or several layman questions of the following paper from eLife dataset. Question: {question}\nBelow is a paper.\n{context}\nNow the paper ends.",
    "prompt_input_exp": "Please answer one or several expert questions of the following paper from eLife dataset. Question: {question}\nBelow is a paper.\n{context}\nNow the paper ends."
}

plos_query_prompt = {
    "prompt_input_pls": "Please answer one or several layman questions of the following paper from PLOS dataset. Question: {question}\nBelow is a paper.\n{context}\nNow the paper ends.",
    "prompt_input_exp": "Please answer one or several expert questions of the following paper from PLOS dataset. Question: {question}\nBelow is a paper.\n{context}\nNow the paper ends."
}


# query generation:
# elife_qg_prompt = {
#     "prompt_input_pls": "Please propose 10 layman questions that are diverse and cover all parts of the following paper from eLife dataset, and then give me the answer of each question.\nBelow is a paper.\n{context}\nNow the paper ends.",
#     "prompt_input_exp": "Please propose 10 expert questions that are diverse and cover all parts of the following paper from eLife dataset, and then give me the answer of each question.\nBelow is a paper.\n{context}\nNow the paper ends."
# }
elife_qg_prompt = {
    "prompt_input_pls": "Please propose 10 layman questions that are diverse and cover all parts of the following paper from eLife dataset.\nBelow is a paper.\n{context}\nNow the paper ends.",
    "prompt_input_exp": "Please propose 10 expert questions that are diverse and cover all parts of the following paper from eLife dataset.\nBelow is a paper.\n{context}\nNow the paper ends."
}

plos_qg_prompt = {
    "prompt_input_pls": "Please propose 10 layman questions that are diverse and cover all parts of the following paper from PLOS dataset.\nBelow is a paper.\n{context}\nNow the paper ends.",
    "prompt_input_exp": "Please propose 10 expert questions that are diverse and cover all parts of the following paper from PLOS dataset.\nBelow is a paper.\n{context}\nNow the paper ends."
}

In [219]:
# def apply_prompt_elife_pls_template(sample):
#     return {
#         "instruction": elife_prompt['prompt_input_pls'].format(context=sample['article'].replace('\n',' ').strip()),
#         "input": "",
#         "output": merge_spaces(sample["plain language summary"].replace('\n',' ')).strip(),
#         "history": []
#     }
# def apply_prompt_elife_exp_template(sample):
#     return {
#         "instruction": elife_prompt['prompt_input_exp'].format(context=sample['article'].replace('\n',' ').strip()),
#         "input": "",
#         "output": merge_spaces(sample["abstract"].replace('\n',' ')).strip(),
#         "history": []
#     }

# def apply_prompt_plos_pls_template(sample):
#     return {
#         "instruction": plos_prompt['prompt_input_pls'].format(context=merge_spaces(sample['article'].replace('\n',' ')).strip()),
#         "input": "",
#         "output": merge_spaces(sample["plain language summary"].replace('\n',' ')).strip(),
#         "history": []
#     }
# def apply_prompt_plos_exp_template(sample):
#     return {
#         "instruction": plos_prompt['prompt_input_exp'].format(context=merge_spaces(sample['article'].replace('\n',' ')).strip()),
#         "input": "",
#         "output": merge_spaces(sample["abstract"].replace('\n',' ')).strip(),
#         "history": []
#     }
def apply_prompt_elife_pls_template(article,summary):
    return {
        "instruction": elife_prompt['prompt_input_pls'].format(context=article.replace('\n',' ').strip()),
        "input": "",
        "output": merge_spaces(summary.replace('\n',' ')).strip(),
        "history": []
    }
def apply_prompt_elife_exp_template(article,summary):
    return {
        "instruction": elife_prompt['prompt_input_exp'].format(context=article.replace('\n',' ').strip()),
        "input": "",
        "output": merge_spaces(summary.replace('\n',' ')).strip(),
        "history": []
    }

def apply_prompt_plos_pls_template(article,summary):
    return {
        "instruction": plos_prompt['prompt_input_pls'].format(context=merge_spaces(article.replace('\n',' ')).strip()),
        "input": "",
        "output": merge_spaces(summary.replace('\n',' ')).strip(),
        "history": []
    }
def apply_prompt_plos_exp_template(article,summary):
    return {
        "instruction": plos_prompt['prompt_input_exp'].format(context=merge_spaces(article.replace('\n',' ')).strip()),
        "input": "",
        "output": merge_spaces(summary.replace('\n',' ')).strip(),
        "history": []
    }

In [213]:

def apply_prompt_elife_query_pls_template(context,question,answer):
    return {
        "instruction": elife_query_prompt['prompt_input_pls'].format(context=context.replace('\n',' ').strip(),question=question),
        "input": "",
        "output": "Answer: " + answer.replace('\n',' ').strip(),
        "history": []
    }
def apply_prompt_elife_query_exp_template(context,question,answer):
    return {
        "instruction": elife_query_prompt['prompt_input_exp'].format(context=context.replace('\n',' ').strip(),question=question),
        "input": "",
        "output": "Answer: " + answer.replace('\n',' ').strip(),
        "history": []
    }

def apply_prompt_plos_query_pls_template(context,question,answer):
    return {
        "instruction": plos_query_prompt['prompt_input_pls'].format(context=context.replace('\n',' ').strip(),question=question),
        "input": "",
        "output": "Answer: " + answer.replace('\n',' ').strip(),
        "history": []
    }
def apply_prompt_plos_query_exp_template(context,question,answer):
    return {
        "instruction": plos_query_prompt['prompt_input_exp'].format(context=context.replace('\n',' ').strip(),question=question),
        "input": "",
        "output": "Answer: " + answer.replace('\n',' ').strip(),
        "history": []
    }

In [ ]:

def apply_prompt_elife_qg_pls_template(context,qa_pairs):
    return {
        "instruction": elife_qg_prompt['prompt_input_pls'].format(context=context.replace('\n',' ').strip()),
        "input": "",
        "output": qa_pairs.strip(),
        "history": []
    }
def apply_prompt_elife_qg_exp_template(context,qa_pairs):
    return {
        "instruction": elife_qg_prompt['prompt_input_exp'].format(context=context.replace('\n',' ').strip()),
        "input": "",
        "output": qa_pairs.strip(),
        "history": []
    }

def apply_prompt_plos_qg_pls_template(context,qa_pairs):
    return {
        "instruction": plos_qg_prompt['prompt_input_pls'].format(context=merge_spaces(context.replace('\n',' ')).strip()),
        "input": "",
        "output": qa_pairs.strip(),
        "history": []
    }
def apply_prompt_plos_qg_exp_template(context,qa_pairs):
    return {
        "instruction": plos_qg_prompt['prompt_input_exp'].format(context=merge_spaces(context.replace('\n',' ')).strip()),
        "input": "",
        "output": qa_pairs.strip(),
        "history": []
    }

In [373]:

from tqdm import tqdm


pdata_lists = []
pdata_qa_lists = []
# elife_sets, plos_sets 
# 
sorted_datas = sorted(plos_datas, key=lambda x: len(merge_spaces(x['article'].replace('\n',' ')).split(' ')))[:3000]
indexed_datas = {sorted_data["doi"]:sorted_data for sorted_data in sorted_datas}
# for data in clip_plos_datas:
#     data_lists.append(apply_prompt_plos_query_exp_template(data))
#     data_lists.append(apply_prompt_plos_query_pls_template(data))
# for data in clip_elife_datas:
#     data_lists.append(apply_prompt_elife_query_exp_template(data))
#     data_lists.append(apply_prompt_elife_query_pls_template(data))
# for plos_pair in plos_exp_query_pairs:
#     if 
# list(elife_sets)[0],list(plos_sets)[0]

for nid, plos_id in enumerate(tqdm(list(plos_sets)[:1500])):
    exp_pairs = plos_exp_query_pairs[plos_id]
    pls_pairs = plos_pls_query_pairs[plos_id]
    
    # context = merge_spaces(sorted_datas[plos_id]['article'].replace('\n',' ')).strip()
    context = merge_spaces(indexed_datas[plos_id]['article'].replace('\n',' ')).strip()
    lay = merge_spaces(indexed_datas[plos_id]['plain language summary'].replace('\n',' ')).strip()
    expert = merge_spaces(indexed_datas[plos_id]['abstract'].replace('\n',' ')).strip()
    
    pdata_lists.append(apply_prompt_plos_exp_template(context,expert))
    # data_lists.append(apply_prompt_plos_pls_template(context,lay))
    
    if nid < 100:
        for exp_pair in exp_pairs[:10]:
            question, answer = exp_pair[0].replace("Question:","").strip(), exp_pair[1].replace("Answer:","").strip()
            pdata_qa_lists.append(apply_prompt_plos_query_exp_template(context,question,answer))
        # data_lists.append(apply_prompt_plos_exp_template(context,expert))
    
    pdata_lists.append(apply_prompt_plos_pls_template(context,lay))
    
    if nid < 100:
        for pls_pair in pls_pairs[:10]:
            question, answer = pls_pair[0].replace("Question:","").strip(), pls_pair[1].replace("Answer:","").strip()
            pdata_qa_lists.append(apply_prompt_plos_query_pls_template(context,question,answer))
        # data_lists.append(apply_prompt_plos_pls_template(context,lay))
        

100%|██████████| 1500/1500 [00:05<00:00, 279.89it/s]


In [336]:
sorted_datas = sorted(elife_datas, key=lambda x: len(merge_spaces(x['article'].replace('\n',' ')).split(' ')))[:2000]
indexed_datas = {sorted_data["doi"]:sorted_data for sorted_data in sorted_datas}

In [337]:
list(indexed_datas.keys())[0]

'elife-39356-v1'

In [380]:
edata_lists = []
edata_qa_lists = []

sorted_datas = sorted(elife_datas, key=lambda x: len(merge_spaces(x['article'].replace('\n',' ')).split(' ')))[:2000]
indexed_datas = {sorted_data["doi"]:sorted_data for sorted_data in sorted_datas}
enum = 0

for nid, elife_id in enumerate(list(indexed_datas.keys())[:1500]):
    
    exp_pairs , pls_pairs = [],[]
    
    if elife_id in list(elife_sets):
        exp_pairs = elife_exp_query_pairs[elife_id]
        pls_pairs = elife_pls_query_pairs[elife_id]
        enum += 1
    
    # context = merge_spaces(sorted_datas[plos_id]['article'].replace('\n',' ')).strip()
    context = merge_spaces(indexed_datas[elife_id]['article'].replace('\n',' ')).strip()
    lay = merge_spaces(indexed_datas[elife_id]['plain language summary'].replace('\n',' ')).strip()
    expert = merge_spaces(indexed_datas[elife_id]['abstract'].replace('\n',' ')).strip()
    
    edata_lists.append(apply_prompt_elife_exp_template(context,expert))
    
    if enum <= 100:
        for exp_pair in exp_pairs[:10]:
            question, answer = exp_pair[0].replace("Question:","").strip(), exp_pair[1].replace("Answer:","").strip()
            edata_qa_lists.append(apply_prompt_elife_query_exp_template(context,question,answer))
            # data_lists.append(apply_prompt_elife_exp_template(context,expert))
    
    edata_lists.append(apply_prompt_elife_pls_template(context,lay))
    
    if enum <= 100:
        for pls_pair in pls_pairs[:10]:
            question, answer = pls_pair[0].replace("Question:","").strip(), pls_pair[1].replace("Answer:","").strip()
            edata_qa_lists.append(apply_prompt_elife_query_pls_template(context,question,answer))
            

In [396]:
len(edata_qa_lists)

2000

In [397]:
# list1 = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
# list2 = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']


length = min(len(pdata_lists), len(pdata_qa_lists))


mixed_list = []


for i in range(0, length, 4):
    mixed_list.extend(pdata_lists[i:i+4])
    mixed_list.extend(pdata_qa_lists[i:i+4])
    mixed_list.extend(edata_lists[i:i+4])
    mixed_list.extend(edata_qa_lists[i:i+4])

mixed_list.extend(pdata_lists[length:])
mixed_list.extend(edata_lists[length:])

# print(mixed_list)

In [415]:
len(mixed_list)

10000

In [414]:
file_path = 'train_query_c3_alpaca.json'

with open(file_path, 'w') as file:
    json.dump(mixed_list, file)

JSON 文件写入成功！


In [18]:
def merge_spaces(input_string):
    output_string = re.sub(r'\s+', ' ', input_string)
    return output_string

In [30]:
clip_plos_datas = []
for data in plos_datas:
    train_text = merge_spaces(data['article'].replace('\n',' ')).strip()
    if len(train_text.split(' ')) <= 5000:
        clip_plos_datas.append(data)

In [31]:
len(clip_plos_datas)

10121

In [15]:
clip_elife_datas = []
for data in elife_datas:
    train_text = data['article'].replace('\n',' ').strip()
    if len(train_text.split(' ')) <= 10000:
        clip_elife_datas.append(data)

In [16]:
len(clip_elife_datas)

2347

In [32]:
data_lists = []

for data in clip_plos_datas:
    data_lists.append(apply_prompt_plos_exp_template(data))
    data_lists.append(apply_prompt_plos_pls_template(data))
for data in clip_elife_datas:
    data_lists.append(apply_prompt_elife_exp_template(data))
    data_lists.append(apply_prompt_elife_pls_template(data))

In [33]:
len(data_lists)

24936

In [34]:
data_lists[1]

{'instruction': 'Please give me a layman summary of the following paper from PLOS dataset.\nBelow is a paper.\nEbola virus belongs to the Filoviridae family of enveloped viruses and contains a non-segmented negative-strand RNA genome [1,2]. Infection in humans can cause Ebola hemorrhagic fever, with exceptionally high case-fatality rates of more than 50% [3,4]. The incubation period of Ebola virus disease (EVD) is 2 to 21 days [5]. The clinical signs and symptoms are extremely similar to those of the Marburg virus and include fever, body aches, vomiting, diarrhea, rash and, in some cases, both internal and external bleeding [5]. Patients usually die of multiple-organ failure or hypovolemic shock. No licensed therapeutic or prophylactic treatments are currently available. The largest outbreak of EVD has been ongoing in West Africa since December 2013. As of April 15, 2015, 25,826 cases (10,704 deaths [41.4%]) had been reported by the World Health Organization (WHO) [6]. Although direct 

In [35]:
import random
random.shuffle(data_lists)

In [42]:
data_lists[4999]

{'instruction': "Please give me an expert summary of the following paper from eLife dataset.\nBelow is a paper.\n1 Introduction Poly ( ADP-ribose ) polymerase 1 ( PARP1 ) serves as a first responder to DNA damage and is the founding member and most abundant representative of the large family of diphtheria toxin-like ADP-ribosyltransferases ( ARTDs ) ( Bai , 2015; Bock and Chang , 2016; Beck et al . , 2014; Daniels et al . , 2015; De Vos et al . , 2012; Mashimo et al . , 2014; Morales et al . , 2014 ) . Binding to either single or double-strand DNA breaks ( SSBs or DSBs ) enzymatically activates PARP1 to use NAD+ in polymerizing long chains of poly ( ADP ) -ribose ( PAR ) onto itself and other nuclear acceptor proteins such as histones and DNA repair proteins . These PAR chains then recruit the appropriate DNA repair machinery containing PAR-binding motifs ( Karlberg et al . , 2013; Teloni and Altmeyer , 2016 ) . PARP1 is of special interest because it is a validated target for cancer t

In [320]:
file_path = 'train_query_alpaca.json'

with open(file_path, 'w') as file:
    json.dump(data_lists, file)

JSON 文件写入成功！


In [137]:
import re
with open("plos_exp_query.txt") as f:
    for line in f.readlines()[:1]:
        print(line)
        # line = re.sub(r'\b\d+:', '\nQuestion:', line.split('\t')[-1]).replace('Answer:'," Answer:").strip()
        # with open('./filter_doc.txt', 'a+', encoding='utf-8') as f:
        #     # print(line.replace(' Answer:',"\nAnswer:"))
        #     f.write(line.replace(' Answer:',"\nAnswer:") + '\n\n')

10.1371/journal.pgen.0040034	1: What are two genetic engineering strategies that can disrupt the expression of a gene?Answer: Targeted recombination and viral integration of gene trap cassettes.2: Who were the recipients of the Nobel Prize in Physiology or Medicine for their pioneering work in targeted recombination mutagenesis in mammals?Answer: Capecchi, Evans, and Smithies.3: When was the discovery of microRNAs made?Answer: Nearly a decade ago.4: What are microRNAs classified as?Answer: MicroRNAs are classified as a class of non-coding genes.5: How many microRNAs are predicted to exist in the mouse genome?Answer: Over 1000 microRNAs are predicted to exist in the mouse genome.6: Where are more than 50% of known microRNAs located?Answer: More than 50% of known microRNAs are located within introns of coding genes.7: What did the researchers investigate in relation to knockout mouse models?Answer: The possibility of coincidentally deleting or disrupting intronic microRNAs in these mouse

In [180]:
sorted_datas = sorted(plos_datas, key=lambda x: len(merge_spaces(x['article'].replace('\n',' ')).split(' ')))

In [181]:
sorted_datas[0]

{'doi': '10.1371/journal.pgen.0040034',
 'title': 'Unintentional miRNA Ablation Is a Risk Factor in Gene Knockout Studies: A Short Report',
 'abstract': 'One of the most powerful techniques for studying the function of a gene is to disrupt the expression of that gene using genetic engineering strategies such as targeted recombination or viral integration of gene trap cassettes. The tremendous utility of these tools was recognized this year with the awarding of the Nobel Prize in Physiology or Medicine to Capecchi, Evans, and Smithies for their pioneering work in targeted recombination mutagenesis in mammals. Another noteworthy discovery made nearly a decade ago was the identification of a novel class of non-coding genes called microRNAs. MicroRNAs are among the largest known classes of regulatory elements with more than 1000 predicted to exist in the mouse genome. Over 50% of known microRNAs are located within introns of coding genes. Given that currently about half of the genes in mou

In [1]:
plos_exp_query_pairs = {}
plos_pls_query_pairs = {}

from collections import defaultdict

In [3]:
# sum = 0
with open("plos_pls_query.txt") as f:
    
    for nid, cline in enumerate(f.readlines()):
        
        # cur_dict = {}
        
        assert sorted_datas[nid]['doi'] == cline.split('\t')[0]
        
        if cline.split('\t')[-1][:2] == "1.":
            vline = "1:" + cline.split('\t')[-1][2:]
        else:
            vline = cline.split('\t')[-1]
            
        for i in range(2,11):
            vline = vline.replace(f".{i}. ", f".{i}: ").replace(f"{i}: ", f".{i}: ")
        # print(vline)
        
        line = re.sub(r'\b\d+: ', '\nQuestion: ', vline).replace('Answer:'," Answer:").strip().replace(' Answer:',"\nAnswer:")
        # with open('./filter_doc.txt', 'a+', encoding='utf-8') as f:
        # print(cline)
        # print(len(line.split('\n')))
        
        # assert len(line.split('\n')) % 2 == 0, cline
        if  len(line.split('\n')) != 20:
            # print(cline,line.split('\n'))
            continue
        
        # sum +=1
        my_list = line.split('\n')
        paired_list = [(my_list[i].strip(), re.sub(r'\.{2,}$', '.', my_list[i+1].strip())) for i in range(0, len(my_list), 2)]
        
        # cur_dict[merge_spaces(sorted_datas[nid]['article'].replace('\n',' ')).strip()] = paired_list
        # cur_dict[cline.split('\t')[0]] = paired_list
        
        plos_pls_query_pairs[cline.split('\t')[0]] = paired_list
        
        # print(paired_list)

NameError: name 'sorted_datas' is not defined

In [1]:
len(plos_pls_query_pairs.keys())

NameError: name 'plos_pls_query_pairs' is not defined

In [189]:
sorted_datas = sorted(elife_datas, key=lambda x: len(x['article'].replace('\n',' ').strip().split(' ')))

In [190]:
sorted_datas[0]

{'doi': 'elife-39356-v1',
 'abstract': 'Tissue mechanics is important for development; however , the spatio-temporal dynamics of in vivo tissue stiffness is still poorly understood . We here developed tiv-AFM , combining time-lapse in vivo atomic force microscopy with upright fluorescence imaging of embryonic tissue , to show that during development local tissue stiffness changes significantly within tens of minutes . Within this time frame , a stiffness gradient arose in the developing Xenopus brain , and retinal ganglion cell axons turned to follow this gradient . Changes in local tissue stiffness were largely governed by cell proliferation , as perturbation of mitosis diminished both the stiffness gradient and the caudal turn of axons found in control brains . Hence , we identified a close relationship between the dynamics of tissue mechanics and developmental processes , underpinning the importance of time-resolved stiffness measurements .',
 'plain language summary': 'Neurons in t

In [191]:
elife_exp_query_pairs = {}
elife_pls_query_pairs = {}

In [194]:
with open("elife_pls_query.txt") as f:
    for nid, cline in enumerate(f.readlines()):
        
        # cur_dict = {}
        
        assert sorted_datas[nid]['doi'] == cline.split('\t')[0]
        
        if cline.split('\t')[-1][:2] == "1.":
            vline = "1:" + cline.split('\t')[-1][2:]
        else:
            vline = cline.split('\t')[-1]
            
        for i in range(2,11):
            vline = vline.replace(f".{i}. ", f".{i}: ").replace(f"{i}: ", f".{i}: ")
        # print(vline)
        
        line = re.sub(r'\b\d+: ', '\nQuestion: ', vline).replace('Answer:'," Answer:").strip().replace(' Answer:',"\nAnswer:")
        # with open('./filter_doc.txt', 'a+', encoding='utf-8') as f:
        # print(cline)
        # print(len(line.split('\n')))
        
        # assert len(line.split('\n')) % 2 == 0, cline
        if  len(line.split('\n')) != 20:
            # print(cline,line.split('\n'))
            continue
        
        sum +=1
        my_list = line.split('\n')
        paired_list = [(my_list[i].strip(), re.sub(r'\.{2,}$', '.', my_list[i+1].strip())) for i in range(0, len(my_list), 2)]
        
        # cur_dict[merge_spaces(sorted_datas[nid]['article'].replace('\n',' ')).strip()] = paired_list
        # cur_dict[cline.split('\t')[0]] = paired_list
        elife_pls_query_pairs[cline.split('\t')[0]] = paired_list
        
        # elife_exp_query_pairs .append(cur_dict)

In [202]:
elife_sets = set(elife_exp_query_pairs.keys()).intersection(set(elife_pls_query_pairs.keys()))
plos_sets = set(plos_exp_query_pairs.keys()).intersection(set(plos_pls_query_pairs.keys()))

In [286]:
sum,max = 0, 0

for k,v in plos_pls_query_pairs.items():
    for q,a in v:
        if max <= len(a.split(' ')):
            max = len(a.split(' '))
        sum += len(a.split(' '))

In [287]:
max, sum/(10*len(plos_pls_query_pairs.keys()))

(62, 16.617878930079776)

In [311]:
len(set(elife_exp_query_pairs.keys()).intersection(set(elife_pls_query_pairs.keys())))

995